In [22]:
:e Arrows
:e QuasiQuotes
import Control.Arrow
import Control.Funflow
import Control.Funflow.Exec.Simple (withSimpleLocalRunner)
import Data.Default
import Path

-- | Flow which asks for the user's name, then their favourite number.
--   The favourite number will be cached, such that if the user comes back,
--   they will not be asked again.
favouriteNumber :: SimpleFlow () (String, String)
favouriteNumber = proc () -> do
    name <- step (\_ -> "Hey you ... ") -< ()
    -- We enable caching for this step. The default cacher uses 'Store' instances
    -- to provide serialisation/deserialisation. The ident is used to ensure that
    -- multiple different steps with the same input do not resolve to the same
    -- cache item.
    fnum <- step' (def { cache = defaultCacherWithIdent 172652 })
                  (\_ -> "What's up?") -< ()
    returnA -< (name, fnum)

-- Use /tmp/funflow as the path to our content store.
-- | Runs the 'favourite number' flow in IO.
runFavouriteNumber :: IO ()
runFavouriteNumber = do
  -- Use /tmp/funflow as the path to our content store.
  res <- withSimpleLocalRunner [absdir|/tmp/funflow|] $ \run ->
    -- 'run' takes the flow and the initial input. Since this flow has an input type of '()',
    -- this is what we provide.
    run favouriteNumber ()
  case res of
    Left err -> putStrLn $ "Something went wrong: " ++ show err
    Right (name, num) -> putStrLn $ name ++ num
    
runFavouriteNumber

[2019-01-24 17:38:01][FFExecutorD.executeLoop][Info][matto-dell][17815][ThreadId 560][funflow-1.4.0-KLdImYfU5mYAAbPXU2bOEB:Control.Funflow.External.Executor src/Control/Funflow/External/Executor.hs:181:9] Initialising connection to coordinator.
Hey you ... What's up?